In [7]:
import glob, os
import pandas as pd

In [13]:
!cd ../../Dropbox/datasets/allerhand

/Users/melvinwevers/source/ah_xml


In [21]:
allFiles = glob.glob('../../Dropbox/datasets/Allerhande_1954-1973/allerhande_processed/data/**/*.txt', recursive=True)
data = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, index_col=None, header=None)
    df['filename'] = os.path.basename(file_)
    list_.append(df)
data = pd.concat(list_)

In [22]:
data['year'] = data['filename'].str[:4]
data['month'] = data['filename'].str[5:7]
data['article'] = data['filename'].str[8:-4]
data.columns.values[0] = 'text'
data = data.drop_duplicates('text', keep='first')
data['text_clean'] = data['text'].str.replace('¬\n', '')
data['text_clean'] = data['text_clean'].str.replace('\n', ' ')
#remove short articles less than 10 characters / words
#data[data['text_clean'].str.len() > 10]
data = data[data['text_clean'].str.count(' ') > 10]
data = data.sort_values(by=['year', 'month'])
data['text_clean'] = data['text_clean'].str.strip()
data = data.drop('text', axis=1)
data = data.reset_index()

In [19]:
data.head(5)

,filename,year,month,article,text_clean
4,1954-12_11.txt,1954,12,11,"Glas voor port, Fliite voor sherry, vermoutb c..."
6,1954-12_11.txt,1954,12,11,l-Tiets is zozeer geschikt om tot de stemming ...
7,1954-12_11.txt,1954,12,11,dat in het bijzonder bij rode wijn nogal eens ...
12,1954-12_11.txt,1954,12,11,Portugese Sardines in olie...... p. blik 38- 5...
16,1954-12_11.txt,1954,12,11,PORT Cockburn's Port (red & white) Conqueror P...


In [23]:
#get id for text (second part) that start with a lowercase character this is due to an incorrect separation or ocr
#and id for text part before that part, probably the first part of the split article (first part)

first_part = data[data['text_clean'].str.split().str.get(0).apply(lambda x: x.islower())].index - 1
second_part = data[data['text_clean'].str.split().str.get(0).apply(lambda x: x.islower())].index

In [26]:
data[data['text_clean'].str.split().str.get(0).apply(lambda x: x.islower())].index

Int64Index([    2,    25,    27,    30,    31,    32,    34,    36,    50,
               52,
            ...
            36222, 36225, 36227, 36229, 36231, 36234, 36236, 36237, 36239,
            36241],
           dtype='int64', length=9630)

In [27]:
#combine these two text parts
text = []
for index, row in data.iterrows():
    if index in second_part:
        index_2 = index - 1
        text.append(data.iloc[index_2]['text_clean'] + data.iloc[index]['text_clean'])

In [29]:
#replace existing text with joined articles. 
data2 = data.ix[first_part] 
data2 = data2.drop('text_clean', axis=1)
data2['text_clean'] = text

/Users/melvinwevers/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [32]:
#remove split articles from data
data = data[~data.index.isin(first_part)]
data = data[~data.index.isin(second_part)]

In [33]:
data3 = pd.concat([data, data2])
data3 = data3.reset_index()

In [13]:
#repeat same tricks for articles with an enter
first_part2 = data3[data3['text_clean'].str.endswith('¬')].index
second_part2 = data3[data3['text_clean'].str.endswith('¬')].index + 1

In [14]:
text2 = []
for index, row in data3.iterrows():
    if index in first_part2:
        index_2 = index + 1
        text2.append(data3.iloc[index]['text_clean'] + data3.iloc[index_2]['text_clean'])

In [15]:
data4 = data3.ix[first_part2]
data4 = data4.drop('text_clean', axis=1)
data4['text_clean'] = text2

data3 = data3[~data3.index.isin(first_part2)]
data3 = data3[~data3.index.isin(second_part2)]

/Users/melvinwevers/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [16]:
data4 = data4.drop('level_0', axis=1)
data3 = data3.drop('level_0', axis=1)

In [17]:
data5 = pd.concat([data3, data4])
data5 = data5.reset_index()

In [22]:
col_drop = ['level_0', 'index']
data_clean = data5.drop(col_drop, axis=1)

In [24]:
data_clean.to_csv("allerhande_clean.csv")